<h1> dada2 pipeline for Taiwan reads</h1>

We're using the <a href='https://www.bioconductor.org/packages/release/bioc/html/dada2.html'>dada2</a> pipeline here. I'll try following [this tutorial](http://benjjneb.github.io/dada2/tutorial.html) Let's dive in. 

<a id='contents'></a>

[Work Environment](#installpackages)

[Getting reads ready for dada2](#prepReads)

<a id='installpackages'></a>
<h1>Work Environment</h1>


<h3>dada2</h3>

In [1]:
source("https://bioconductor.org/biocLite.R")

Bioconductor version 3.6 (BiocInstaller 1.28.0), ?biocLite for help


In [9]:
biocLite("dada2")

BioC_mirror: https://bioconductor.org
Using Bioconductor 3.6 (BiocInstaller 1.28.0), R 3.4.3 (2017-11-30).
Installing package(s) ‘dada2’
Warning message in install.packages(pkgs = doing, lib = lib, ...):
“installation of package ‘dada2’ had non-zero exit status”installation path not writeable, unable to update packages: codetools, lattice,
  spatial
Old packages: 'Matrix'


In [1]:
library(dada2)

Loading required package: Rcpp


In [11]:
biocLite(ask=FALSE)

BioC_mirror: https://bioconductor.org
Using Bioconductor 3.6 (BiocInstaller 1.28.0), R 3.4.3 (2017-11-30).
installation path not writeable, unable to update packages: codetools, lattice,
  spatial
Updating packages 'Matrix'
Warning message in install.packages(update[instlib == l, "Package"], l, contriburl = contriburl, :
“installation of package ‘Matrix’ had non-zero exit status”Updating HTML index of packages in '.Library'
Warning message in file.create(f.tg):
“cannot create file '/usr/share/R/doc/html/packages.html', reason 'Permission denied'”Warning message in make.packages.html(.Library):
“cannot update HTML package index”

In [13]:
biocValid() 

[1] TRUE

In [2]:
packageVersion("dada2")

[1] ‘1.6.0’

<h3>Set paths to reads</h3>

We have separate directories for wood reads and leaf reads

In [3]:
leafpath <- "/home/daniel/Documents/taiwan/leafreads"

In [4]:
woodpath <- "/home/daniel/Documents/taiwan/woodreads"

In [5]:
head(list.files(woodpath))

[1] "lane1-s160-index-AAGCACTG-GTGATCCANNNN-Dc-X_S160_L001_R1_001.fastq"   
[2] "lane1-s160-index-AAGCACTG-GTGATCCANNNN-Dc-X_S160_L001_R2_001.fastq"   
[3] "lane1-s161-index-AAGCACTG-TTCGTACGNNNN-Dc-PosG_S161_L001_R1_001.fastq"
[4] "lane1-s161-index-AAGCACTG-TTCGTACGNNNN-Dc-PosG_S161_L001_R2_001.fastq"
[5] "lane1-s162-index-AAGCACTG-ATGACAGGNNNN-Dc-PosI_S162_L001_R1_001.fastq"
[6] "lane1-s162-index-AAGCACTG-ATGACAGGNNNN-Dc-PosI_S162_L001_R2_001.fastq"

<a id='prepReads'></a>
<h1>Getting reads ready for dada2</h1>

Before we begin the dada2 pipeline, we need to demultiplex our leaf reads, clean off primers and any other non-biological parts of our sequences, and make sure that forward and reverse reads are in the same order in the fastq files.  

<h3>Demultiplexing</h3>

Wood reads are demultiplexed, but for the leaves are not. They were labeled using split golay barcodes, so we need to reassemble these barcodes, for both the forward and reverse reads. I have a script for this. The latest version is in this repository, BCunsplit.py

But, as I'm not much of a programmer, this script doesn't handle big read sets very well, somehow gets bogged down despite my effort to keep the entire files from being loaded into memory. So let's break up the leaf reads, apply the script, then reassemble. Most of this is in the shell, not R.

In [1]:
pwd

/home/daniel/Documents/taiwan/taiwan_dada2


In [8]:
leafR1=/home/daniel/Documents/taiwan/leafreads/TaiwanFA_R1.fastq
leafR2=/home/daniel/Documents/taiwan/leafreads/TaiwanFA_R2.fastq

In [ ]:
## break up the fastq files, keep things in multiples of four...
split -d -l 10000000 $leafR1 leafR1sub
split -d -l 10000000 $leafR2 leafR2sub

Now cycle through these, apply our script to each:

In [3]:
nus=(00 01 02 03 04 05 06 07)
for i in ${nus[*]}
do
../BCunsplit.py leafR2sub$i leafR1sub$i
echo $i
done

00
01
02
03
04
05
06
07


Combine, take a look:

In [7]:
cat rearranged_leafR1sub0* > rearranged_leafR1.fastq && cat rearranged_leafR2sub0* > rearranged_leafR2.fastq &

In [9]:
## take the first six BP of the first 10 reads from each, 
## check to see if they are the same
aa=$(head -n 40 rearranged_leafR2.fastq | sed -n '2~4p' | cut -c -6)
bb=$(head -n 40 $leafR2 | sed -n '2~4p' | cut -c -6)
echo $aa
echo $bb
if [ "$aa" == "$bb" ]; then echo "true"; fi

CACCTC CTTCCT CGTTAA CCTCCT CCACAT TCTCCA CCTCGC CCCCAT CCTGAT CACCTC
CACCTC CTTCCT CGTTAA CCTCCT CCACAT TCTCCA CCTCGC CCCCAT CCTGAT CACCTC
true


In [10]:
## take the first six BP of the last 10 reads from each, 
## check to see if they are the same
aa=$(tail -n 40 rearranged_leafR2.fastq | sed -n '2~4p' | cut -c -6)
bb=$(tail -n 40 $leafR2 | sed -n '2~4p' | cut -c -6)
echo $aa
echo $bb
if [ "$aa" == "$bb" ]; then echo "true"; fi

AATGAG CCCCTT AATGAT TCGCCT CCGCAT CACCTC CCTCCT CCTCAT ATAATT CTTTTN
AATGAG CCCCTT AATGAT TCGCCT CCGCAT CACCTC CCTCCT CCTCAT ATAATT CTTTTN
true


<h3>Removing Primers</h3>

<h3>Order of reads</h3>